<a href="https://colab.research.google.com/github/baekyongbo/jejumaster/blob/main/3%EC%9D%BC%EC%B0%A8_%EC%9B%B9%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_01_OpenAPI%EB%A5%BC%EC%9D%B4%EC%9A%A9%ED%95%9C%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%88%98%EC%A7%91_%EC%99%84%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 웹 데이터 분석

## 1. OpenAPI를 이용한 데이터 수집

---

### 자신의 PC 폰트 목록 가져오기

In [ ]:
#(Mac) 폰트 목록 가져오기
import matplotlib.font_manager as fm

font_list_mac = fm.OSXInstalledFonts()
print(f'Mac 폰트 목록 : {font_list_mac}')

In [ ]:
#(windows) 폰트 목록 가져오기
import matplotlib.font_manager as fm
font_list_win = fm.findSystemFonts(fontpaths=None, fontext='ttf')
print(f'windows 폰트 목록 : {font_list_win}')

In [ ]:
import matplotlib.pyplot as plt
#(Mac한글 폰트 지정하기)
# plt.rcParams['font.family'] =

#(windows)한글 폰트 지정하기
plt.rcParams['font.family'] = 'Malgun Gothic'

### 라이브러리 설치하기

In [ ]:
# 정적 크롤링을 위한 requests 설치
!pip install requests

### 01.웹 크롤링의 이해
- **웹 크롤링(Web Crawling)**: 웹 페이지로부터 원하는 정보를 추출하는 기법, 보통 여러 페이지 탐색하는 방법,
- **웹 스크래핑(Web Scraping)**: 특정한 하나의 웹 페이지를 탐색하는 방법
- **웹 크롤링과 웹 스크래핑의  방법이 동일하여 보통 웹 크롤링이라고 통칭해서 부른다.**

---------------------

## 02.네이버 OpenAPI 사용하기

### # 1.네이버 OpenAPI 신청하기 : 네이버 **검색(책, 뉴스)

- 네이버 OpenAPI 소개: https://developers.naver.com/products/intro/plan/
- 개발 가이드 보기: https://developers.naver.com/docs/serviceapi/search/news
- OpenAPI 신청하기: https://developers.naver.com/apps/#/register

### # 2.네이버 OpenAPI 사용 예 : 책 검색

### [실습] : 네이버 검색 API 사용하여 데이터 수집하기

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd

now = datetime.datetime.now()
datas = []   #csv파일을 위한 변수

client_id = '자신의 client_id'  # 자신의 client_id
client_pw = '자신의 client_secret'            # 자신의 client_secret


#[CODE 1]
def get_RequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_pw)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print(f"[{now.strftime('%Y년%m월%d일 %H시%M분%S초')}] Url Request Success")
            return response.read().decode('utf-8')

    except Exception as e:
#         if response.getcode() == 400 and datas:
#             return None
        print(e)
        print(f"[{now.strftime('%Y년%m월%d일 %H시%M분%S초')}] Error for URL : {url}" )
        return None


#[CODE 2]  네이버 검색 API
def get_NaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = get_RequestUrl(url)   #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def get_PostData(node, post, jsonResult, cnt):
    if node == 'book':
        data = {'제목':post['title'],
                '저자':post['author'],
                '출판사':post['publisher'],
                '출간일':post['pubdate'],
                '링크':post['link'],
                '이미지':post['image']}
    elif node == 'news':
        data = {'제목':post['title'],
                '링크':post['originallink'],
                '내용':post['description']}
    elif node == 'shop':
        data = {'제목':post['title'],
                '브랜드':post['brand'],
                '제조사':post['maker'],
                '가격':post['lprice'],
                '이미지':post['image']}

    jsonResult.append(data)
    datas.append(data)

    return

#[CODE 0]
def main():
    nodeType = '''-----------------------------
    네이버 검색 대상입니다.
      1.book,  2.news,  3.shop
    -----------------------------'''
    print(nodeType)
    node = input('번호를 선택하세요.')
    if node == '1': node = 'book'
    elif node == '2': node = 'news'
    elif node == '3': node = 'shop'

    srcText = input(f'{node} 검색할 검색어를 입력하세요: ')

    display, cnt = 100, 0
    jsonResult = []
    jsonResponse = get_NaverSearch(node, srcText, 1, display)  #[CODE 2]
    total = jsonResponse['total']

    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            get_PostData(node, post, jsonResult, cnt)  #[CODE 3]

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = get_NaverSearch(node, srcText, start, total)
#     print(f'가져올 데이터 : {total} 건')

    with open(f'./data/naver_{node}_{srcText}', 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))



    # csv 파일로 저장하기
    file = f'./data/naver_{node}_{srcText}.csv'
    df = pd.DataFrame(datas)
    df.to_csv(file, index=False, encoding="utf-8-sig")
    print(f'./data/naver_{node}_{srcText}.csv SAVED')
    return df

#-----------
# 시작
#-----------
df = main()
df

-----------------------------
    네이버 검색 대상입니다.
      1.book,  2.news,  3.shop
    -----------------------------
번호를 선택하세요.3
shop 검색할 검색어를 입력하세요: 여성니트
[2023년10월22일 21시27분55초] Url Request Success
HTTP Error 400: Bad Request
[2023년10월22일 21시27분55초] Error for URL : https://openapi.naver.com/v1/search/shop.json?query=%EC%97%AC%EC%84%B1%EB%8B%88%ED%8A%B8&start=101&display=18760172
가져온 데이터 : 100 건
./data/naver_shop_여성니트.csv SAVED


,제목,브랜드,제조사,가격,이미지
0,부드러운 기본 블랙 핑크 23컬러 가을 간절기 데일리 캐시미어 라운드 <b>여성 니...,,,19100,https://shopping-phinf.pstatic.net/main_811054...
1,루즈핏 반폴라 캐시미어니트 여성 목폴라 니트 반목 폴라 <b>여자니트</b>티 가을...,,,19800,https://shopping-phinf.pstatic.net/main_822705...
2,부드러운 루즈핏 캐시미어니트 간절기 가을 라운드 브이넥니트 긴팔 <b>여성니트</b...,,,18400,https://shopping-phinf.pstatic.net/main_820167...
3,벤시몽 23FW <b>여성</b> 케이블 <b>니트</b> 풀오버 3종,벤시몽,,63400,https://shopping-phinf.pstatic.net/main_423913...
4,<b>여성니트</b>티 비스코스 목폴라니트 하프넥 반폴라니트 캐시미어 반목폴라,,,18800,https://shopping-phinf.pstatic.net/main_106595...
...,...,...,...,...,...
95,나인 로더 부클 자수 블루종 <b>니트</b> T 6537,나인,바이와이제이,48929,https://shopping-phinf.pstatic.net/main_342753...
96,폴로랄프로렌 케이블 <b>니트</b> <b>여자</b> 긴팔 폴로 꽈배기 라운드넥 ...,폴로랄프로렌,폴로랄프로렌,163000,https://shopping-phinf.pstatic.net/main_829268...
97,커버낫 <b>우먼 니트</b> 후디 집업 차콜 CO2304KT64CH,커버낫,,120690,https://shopping-phinf.pstatic.net/main_429238...
98,타미힐피거 <b>여자</b> 꽈배기 케이블 <b>니트</b> 코튼 라운드넥 긴팔 티...,타미힐피거,타미힐피거,68000,https://shopping-phinf.pstatic.net/main_866018...


### [실습] :  네이버 Papago API 사용하여 번역하기

In [ ]:
import os
import sys
import urllib.request
import datetime
import json


# client_id = ''  # 자신의 파파고 NMT API ID
# client_pw = ''            # 자신의 파파고 NMT API PASSWORD

news = []   #csv파일을 위한 변수

#[CODE 1]
def papago(prompt, lang=1):

    encText = urllib.parse.quote(prompt)
    if lang == 1 :
        data = "source=ko&target=en&text=" + encText
    else:
        data = "source=en&target=ko&text=" + encText
    url = "https://openapi.naver.com/v1/papago/n2mt"

    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_pw)

    try:
        response = urllib.request.urlopen(req, data=data.encode("utf-8") )
        if response.getcode() == 200:
            print(f"[{now.strftime('%Y년%m월%d일 %H시%M분%S초')}] Url Request Success")
            responseDecode = response.read().decode('utf-8')

            if responseDecode == None:
                return None
            else:
                return json.loads(responseDecode)

    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

# 메인
lang= int(input('[선택] 1:한글->영어, 2:영어->한글: '))
txt =  '한글' if lang==1 else '영어'
prompt= input(f'[번역] 변환할 문장을 입력하세요[{txt}]')
print(f'# 입력 문자 => {prompt}')

jsonResponse = papago(prompt, lang)  # 파파고 번역
jsonResponse
print('-'*50)
print(f"# 번역 결과 => {jsonResponse['message']['result']['translatedText']}")


[선택] 1:한글->영어, 2:영어->한글: 2
[번역] 변환할 문장을 입력하세요[영어]Data analysis is so much fun.
# 입력 문자 => Data analysis is so much fun.
[2023년10월22일 21시00분12초] Url Request Success
--------------------------------------------------
# 번역 결과 => 데이터 분석은 정말 재미있다.


-------------------------

## 03.웹 이미지 수집하기

### # 1. 웹 이미지 화면에 출력하기

In [ ]:
from IPython.display import Image
Image(url='https://shopping-phinf.pstatic.net/main_3815224/38152244716.20230516165404.jpg')

### # 2.  웹 이미지 파일로 저장하기

In [ ]:
import requests
from io import BytesIO
from PIL import Image

# 다운받을 이미지 url
urls = [
    "https://shopping-phinf.pstatic.net/main_3815224/38152244716.20230516165404.jpg",
    "https://shopping-phinf.pstatic.net/main_3726611/37266116619.20230119071117.jpg"
]
for idx, url in enumerate(urls):
    res = requests.get(url)                   # url 요청
    img = Image.open(BytesIO(res.content))    # 응답결과(res.content)바이트파일 이미지 파일로 오픈
    img.save(f'./testimg_{idx}.png', 'png')   # 'png'이미지로(만) 저장

### [실습] : 웹 이미지 수집하기

In [ ]:
# 이미지 다운로드 하기
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import time
import os

FILE = './data/naver_shop_여성니트.csv'  # 앞에서 자신이 검색한 쇼핑 파일
ImgFolder = './image/download/'         # 다운받을 이미지 폴더
MAX = 10                                # 이미지 파일 다운로드 건수

def getImageUrl(file):
    print(f'읽은 파일명: {file}')
    df = pd.read_csv(file, encoding='utf-8') # 이미지가 있는 쇼핑 파일 불러오기
    return df['이미지'].to_list()

def createDirectory(directory): # 다운받을 이미지 폴더 만들기
    try:
        if not os.path.exists(directory):
            print(f'{directory} 폴더가 생성되었습니다.')
            os.makedirs(directory)
        print(f'이미지 폴더 위치 : {directory}')
    except OSError:
        print("Error: Failed to create the directory.")

def downloadImageFile(urls, imgfolder):
    start = time.time()             # 이미지 다운로드 속도 time check
    for idx, url in enumerate(urls):
        if idx == MAX:
            break  # MAX 건수만 처리하기
        res = requests.get(url)     # request.get 요청
#         print(f'[{idx+1:2>}][{time.time() - start}] : {url}')  # 이미지 다운로드 시간 체크
        print(f'[{idx+1:0>2}] : {url}')  # 이미지 다운로드 시간 체크
        img = Image.open(BytesIO(res.content))  #Img open
        img.save(f'{imgfolder}testimage_{idx}.png', 'png')
    return idx


urls = getImageUrl(FILE)                # 이미지 URL 목록 가져오기
createDirectory(ImgFolder)              # 다운받을 이미지 폴더 만들기
totalcnt = downloadImageFile(urls, imgfolder)      # 이미지 다운로드하기
print(f'총 다운로드 건수: {totalcnt}')


읽은 파일명: ./data/naver_shop_여성니트.csv
이미지 폴더 위치 : ./image/download/
[01] : https://shopping-phinf.pstatic.net/main_8110549/81105494642.jpg
[02] : https://shopping-phinf.pstatic.net/main_8227052/82270521487.3.jpg
[03] : https://shopping-phinf.pstatic.net/main_8201676/82016764964.15.jpg
[04] : https://shopping-phinf.pstatic.net/main_4239133/42391336405.20231014071141.jpg
[05] : https://shopping-phinf.pstatic.net/main_1065955/10659550627.5.jpg
[06] : https://shopping-phinf.pstatic.net/main_4287907/42879071610.20230926080053.jpg
[07] : https://shopping-phinf.pstatic.net/main_4189969/41899691388.20230817034832.jpg
[08] : https://shopping-phinf.pstatic.net/main_8223769/82237690733.jpg
[09] : https://shopping-phinf.pstatic.net/main_3947443/39474430403.20230919013004.jpg
[10] : https://shopping-phinf.pstatic.net/main_8201675/82016750293.1.jpg
총 다운로드 건수: 10


---------------------------

--------

## [주의!!!] 아래 내용은 ChatGPT를 OpenAPI를 이용해서 사용하는 방법으로 각자 사용할 수 있는 API 사용량이 있어야 오류없이 실행 가능하다.

## 02.OpenAI OpenAPI 사용하기(ChatGPT)

### # 1.OpenAI OpenAPI 신청하기

- 참고 : https://passwd.tistory.com/entry/Python-OpenAI-API-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0

        
### 작업 순서
https://platform.openai.com/docs/api-reference/introduction
 1. Introduction(openai 모듈 설치하기) : https://platform.openai.com/docs/libraries/community-libraries
 2. openai API KEY: Authentication(Secret Key 발급받기) : https://platform.openai.com/account/api-keys  </br>
 3. API 코드 작성하기
 4. 자신이 사용한 API 사용량 확인하기: https://platform.openai.com/account/usage

### 1. openai 모듈 설치하기

In [ ]:
!pip install openai

### 2. openai API 신청하기
- https://platform.openai.com/account/api-keys

### 3. API 코드 작성하기

### [실습문제] OpenAI 모델 목록 확인하기

In [ ]:
# API 키 입력하기
YOUR_API_KEY = ""

In [ ]:
import os
import openai

openai.organization = YOUR_ORG
openai.api_key = YOUR_API_KEY
openai.Model.list()

### [실습문제] ChatGPT로 챗봇 만들기

- https://jaeyung1001.tistory.com/entry/Slack-ChatGPT-Slack-%EC%B1%97%EB%B4%87-%EB%A7%8C%EB%93%A4%EA%B8%B0-1%ED%8E%B8

In [ ]:
import os
import openai

def ChatGPT(prompt, ORG=YOUR_ORG, API_KEY=YOUR_API_KEY):
    # api key 세팅
    openai.organization = ORG
    openai.api_key = API_KEY

    # ChatGPT API 호출 및 최신 언어 모델인 text-davinci-003을 가져옴
    response = openai.Completion.create(
        engine='gpt-3.5-turbo'  # text-davinci-003'text-curie-001'  # 'text-babbage-001' #'text-ada-001'
        , prompt=prompt
        , temperature=0.5
        , max_tokens=1024
        , top_p=1
        , frequency_penalty=0
        , presence_penalty=0)

    return response['choices'][0]['text']


def main():
    # 지문 입력 란
    prompt = input("[Question] 질문을 입력해 주세요: ")
    print('[Answer]:  ')
    print(ChatGPT(prompt).strip())


if __name__ == '__main__':
    main()

#### [RateLimitError 오류 예외처리 적용]

- 질문: 재미있는 블로그 이름 50개 추천해줘
- 질문: 방금 알려준 블로그 이름을 제목과 분류기준으로 나누어서 csv 형태로 만들어줘.
- 질문: 웃긴이야기 블로그에 들어갈 블로그 글을 재미있는 예제 2개를 넣어서 markdown 형식으로 1000자 이내로 블로그 글을 작성해줘.

In [ ]:
import os
import sys
import openai


# ChatGPT 사용하기
def ChatGPT(prompt, API_KEY=YOUR_API_KEY):
    # api key 세팅
    openai.api_key = API_KEY

    # ChatGPT API 호출 및 최신 언어 모델인 text-davinci-003을 가져옴
    response = openai.Completion.create(
        engine='gpt-3.5-turbo-0613'  # 'text-davinci-003' 'text-curie-001' 'text-babbage-001' 'text-ada-001'
        , prompt=prompt
        , temperature=0.5
        , max_tokens=1024
        , top_p=1
        , frequency_penalty=0
        , presence_penalty=0)

    return response['choices'][0]['text']

# 메인
def main():
    # 지문 입력 란
    try:
        print("[ChatGPTBot]: 프로그램을 시작합니다.\n\t종료하려면 [myQ]상태에서 q를 입력하세요.")
        print()
        while True:
            prompt = input("[myQ]: ")
            if prompt == 'q':
                print('[ChatGPTBot]: 프로그램을 종료합니다.')
                break

            print()
            print('[ChatGPTBot]:', ChatGPT(prompt).strip() )
            print()
        # sys.exit()

    except Exception as e:  # 예외처리하기
        if str(e).find('RateLimitError'):
            print()
            print("RateLimitError: chatGPT 서버에 연결이 원활하지 않습니다.")
            print("잠시후 다시 시도해 주세요.")
        else:
            print(e)

if __name__ == '__main__':
    main()

### 4. 자신이 사용한 API 사용량 확인하기:
https://platform.openai.com/account/usage

----------------

끝!